In [ ]:
# Cell 1: Importera alla bibliotek vi behöver för u2 (Decision Trees)

import pandas as pd              # Datahantering (DataFrames)
import numpy as np               # Numeriska beräkningar

from sklearn.model_selection import train_test_split    # Dela upp data i train/test
from sklearn.tree import DecisionTreeClassifier         # Beslutsträd (Decision Tree)
from sklearn.preprocessing import MinMaxScaler          # Normalisering till [0,1]
from sklearn.metrics import confusion_matrix, accuracy_score  # Mått på prestanda


In [ ]:
# Cell 2: Läs in datasetet student-mat och skapa features (X) och target (y)

# Läs in student-mat.csv (matematik-elever)
df = pd.read_csv("student-mat.csv", sep=";")

# Välj alla numeriska kolumner i datan
numeric_cols = df.select_dtypes(include=[np.number]).columns

# Features X = alla numeriska kolumner UTOM G3 (slutbetyget)
X = df[numeric_cols].drop(columns=["G3"])

# Target y = slutbetyg G3 omvandlat till tre klasser:
# 0–9   -> "low"
# 10–14 -> "medium"
# 15–20 -> "high"
bins = [-1, 9, 14, 20]                # Gränser för intervallen
labels = ["low", "medium", "high"]    # Klassnamn
y = pd.cut(df["G3"], bins=bins, labels=labels)

# Kolla att vi har rimligt med observationer i varje klass
print("Fördelning av klasser i y:")
print(y.value_counts())
print("\nForm på X (antal rader, antal features):", X.shape)


In [ ]:
# Cell 3: Funktion som gör EN körning med beslutsträd

def run_decision_tree_once(X, y, test_size, normalize=False):
    """
    Gör EN körning med DecisionTreeClassifier.

    Parametrar:
    - X : features (predictors)
    - y : target (klasserna low/medium/high)
    - test_size : andel av datan som ska vara test (t.ex. 0.1 eller 0.3)
    - normalize : True => normalisera X till [0,1] (MinMaxScaler)

    Returnerar:
    - acc : accuracy på testdatan
    - cm  : confusion matrix (3x3, ordning: low, medium, high)
    """

    # 1. Dela upp data i train och test
    # random_state sätts inte => train/test-split slumpas olika varje gång
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=test_size,
        stratify=y          # Behåll samma klassfördelning i train och test
    )

    # 2. Normalisera features om vi valt det
    if normalize:
        scaler = MinMaxScaler(feature_range=(0, 1))
        X_train = scaler.fit_transform(X_train)   # Lär sig min/max från träningsdatan
        X_test = scaler.transform(X_test)         # Använder samma skalning på testdatan
    else:
        # Behåll som numpy-arrayer
        X_train = X_train.values
        X_test = X_test.values

    # 3. Skapa och träna ett beslutsträd
    # random_state=None (default) => trädet kan slumpas olika varje gång
    clf = DecisionTreeClassifier(random_state=None)
    clf.fit(X_train, y_train)

    # 4. Prediktion på testdatan
    y_pred = clf.predict(X_test)

    # 5. Beräkna accuracy och confusion matrix
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred, labels=y.cat.categories)

    return acc, cm


In [ ]:
# Cell 4: Funktion som upprepar ett experiment n_runs gånger

def run_decision_tree_experiment(X, y, test_size, normalize=False, n_runs=100):
    """
    Kör samma konfiguration (samma test_size + normalize) n_runs gånger
    och beräknar statistik.

    Returnerar:
    - mean_acc : medelaccuracy över alla körningar
    - best_acc : bästa (högsta) accuracy
    - best_cm  : confusion matrix för den körning som hade bäst accuracy
    - all_accs : lista med accuracy-värden (en per körning)
    """
    all_accs = []
    best_acc = -1
    best_cm = None

    for i in range(n_runs):
        acc, cm = run_decision_tree_once(X, y, test_size=test_size, normalize=normalize)
        all_accs.append(acc)

        # Spara den bästa körningen
        if acc > best_acc:
            best_acc = acc
            best_cm = cm

    mean_acc = np.mean(all_accs)
    return mean_acc, best_acc, best_cm, all_accs


In [ ]:
# Cell 5: Kör alla experiment (4 st: 2 splittar * 2 datatyper) med 100 körningar vardera

settings = [
    (0.10, False, "train=90%, test=10%, originaldata"),
    (0.10, True,  "train=90%, test=10%, normaliserat [0,1]"),
    (0.30, False, "train=70%, test=30%, originaldata"),
    (0.30, True,  "train=70%, test=30%, normaliserat [0,1]")
]

results = []

for test_size, normalize_flag, description in settings:
    print("===================================================")
    print("Experiment:", description)
    print(f"test_size = {test_size}, normalize = {normalize_flag}")

    # Kör experimentet 100 gånger
    mean_acc, best_acc, best_cm, all_accs = run_decision_tree_experiment(
        X, y,
        test_size=test_size,
        normalize=normalize_flag,
        n_runs=100
    )

    print(f"\nMedelaccuracy över 100 körningar: {mean_acc:.4f}")
    print(f"Bästa accuracy av 100 körningar: {best_acc:.4f}")
    print("Confusion matrix för bästa körningen (low/medium,high):")
    print(best_cm)

    # Spara resultat för tabell senare
    results.append({
        "description": description,
        "test_size": test_size,
        "normalize": normalize_flag,
        "mean_accuracy": mean_acc,
        "best_accuracy": best_acc,
        "best_confusion_matrix": best_cm
    })


In [ ]:
# Cell 6: Sammanställ resultat i en tabell (liknande Figur A2)

res_df = pd.DataFrame([
    {
        "Beskrivning": r["description"],
        "Test_size": r["test_size"],
        "Normaliserat": r["normalize"],
        "Medelaccuracy": r["mean_accuracy"],
        "Bästa accuracy": r["best_accuracy"]
    }
    for r in results
])

# Sortera efter Medelaccuracy, högst först
res_df_sorted = res_df.sort_values(by="Medelaccuracy", ascending=False)

print("Resultat för alla experiment (sorterat efter medelaccuracy):")
display(res_df_sorted)


In [ ]:
# Cell 7 (valfri): Visa confusion matrix för experimentet med högst medelaccuracy

best_row = res_df_sorted.iloc[0]  # första raden (högst medelaccuracy)
best_desc = best_row["Beskrivning"]

print("Bästa experiment enligt medelaccuracy:")
print(best_desc)

# Hitta motsvarande dict i results-listan
for r in results:
    if r["description"] == best_desc:
        print("\nConfusion matrix för bästa körningen i detta experiment:")
        print(r["best_confusion_matrix"])
        break
